In [1]:
import scanpy as sc
import pandas as pd
import anndata as ad
import anndata2ri
import os
from os.path import join
import scipy.sparse as sp
import numpy as np

from hlca_v2.ingestion_utils import get_gspread_df, ValidationWorkflow, AnnDataMerger

GSPREAD_JSON = "/home/icb/raphael.kfuri-rubens/data/hlca_v2/google_sheets_api/hlca-v2-8d5fea10d8f3.json"

In [2]:
DATASET_ID = "Tabula_Sapiens_2022_publ"
H5AD_PATH = f"/home/icb/raphael.kfuri-rubens/data/hlca_v2/{DATASET_ID}/{DATASET_ID}.h5ad"
OUTPUT_PATH_PREREVISION = '/home/icb/raphael.kfuri-rubens/data/hlca_v2/HLCA_V2_CORE/adata_prerevision'
OUTPUT_PATH_POSTREVISION = '/home/icb/raphael.kfuri-rubens/data/hlca_v2/HLCA_V2_CORE/adata_postrevision'

In [3]:
# Name constants
AUTHOR_CELL_TYPE_L0 = 'author_cell_type_level_0'
AUTHOR_CELL_TYPE_L1 = 'author_cell_type_level_1'

CELL_TYPE_ONTOLOGY_ID_L0 = 'cell_type_ontology_term_id_level_0'
CELL_TYPE_ONTOLOGY_ID_L1 = 'cell_type_ontology_term_id_level_1'

CELL_TYPE_ONTOLOGY_LABEL_L0 = 'cell_type_ontology_term_label_level_0'
CELL_TYPE_ONTOLOGY_LABEL_L1 = 'cell_type_ontology_term_label_level_1'

AUTHOR_CELL_TYPE_DESCRIPTION_L0 = 'author_cell_type_description_level_0'
AUTHOR_CELL_TYPE_DESCRIPTION_L1 = 'author_cell_type_description_level_1'

MARKER_GENES_L0 = 'author_cell_type_markers_level_0'
MARKER_GENES_L1 = 'author_cell_type_markers_level_1'

# Finest grained annotation will be generic dataset cell type
AUTHOR_CELL_TYPE = 'author_cell_type'
CELL_TYPE_ONTOLOGY_ID = 'cell_type_ontology_term_id'
CELL_TYPE_ONTOLOGY_LABEL = 'cell_type_ontology_term_label'
MARKER_GENES = 'author_cell_type_markers'
AUTHOR_CELL_TYPE_DESCRIPTION = 'author_cell_type_description'

# Load data

In [4]:
adata = sc.read_h5ad(H5AD_PATH)
obs = get_gspread_df(GSPREAD_JSON, DATASET_ID, "tier_1", "obs")
uns = get_gspread_df(GSPREAD_JSON, DATASET_ID, "tier_1", "uns")

# Validate obs and uns from Tier 1 Metadata Template

In [5]:
val_workflow = ValidationWorkflow(
    input = uns,
    axis = 'uns'
)

validated_uns = val_workflow.init_workflow()
validated_uns

Validation workflow successful. Returning UNS dataframe.

,title,study_PI,batch_condition,default_embedding,unpublished,comments
0,Tabula Sapiens 2022,Stephen Quake,"donor,method (batch correction performed for l...",X_scvi,published,


In [6]:
val_workflow = ValidationWorkflow(
    input = obs,
    axis = 'obs'
)

validated_obs = val_workflow.init_workflow()
validated_obs

Validation workflow successful. Returning OBS dataframe.

,sample_ID,donor_id,protocol_URL,institute,sample_collection_site,sample_collection_relative_time_point,library_ID,library_ID_repository,author_batch_notes,organism_ontology_term_id,...,sequenced_fragment,sequencing_platform,is_primary_data,reference_genome,gene_annotation_version,alignment_software,intron_inclusion,disease_ontology_term_id,self_reported_ethnicity_ontology_term_id,development_stage_ontology_term_id
0,Donor 1,TSP1,https://www.science.org/doi/10.1126/science.ab...,Stanford University/UCSF/CZ Biohub SF,N/A,N/A,10X_library column in obs or cDNA plate for sm...,N/A (data available in AWS),N/A,NCBITaxon:9606,...,3 prime tag,EFO:0008637,true,GRCh38,N/A,cell ranger 5.0.1,no,PATO:0000461,HANCESTRO:0005,HsapDv:0000240
1,Donor 2,TSP2,https://www.science.org/doi/10.1126/science.ab...,Stanford University/UCSF/CZ Biohub SF,N/A,N/A,10X_library column in obs or cDNA plate for sm...,N/A (data available in AWS),N/A,NCBITaxon:9606,...,3 prime tag,EFO:0008637,true,GRCh38,N/A,cell ranger 5.0.1,no,PATO:0000461,HANCESTRO:0016,HsapDv:0000241
2,Donor 14,TSP14,https://www.science.org/doi/10.1126/science.ab...,Stanford University/UCSF/CZ Biohub SF,N/A,N/A,10X_library column in obs or cDNA plate for sm...,N/A (data available in AWS),N/A,NCBITaxon:9606,...,3 prime tag,EFO:0008637,true,GRCh38,N/A,cell ranger 5.0.1,no,PATO:0000461,HANCESTRO:0005,HsapDv:0000240


In [7]:
# Subset to lung only
adata = adata[adata.obs['tissue'] == 'lung']
adata

View of AnnData object with n_obs × n_vars = 35682 × 58482
    obs: 'tissue_in_publication', 'assay_ontology_term_id', 'donor_id', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'sex_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'suspension_type', 'cell_type_ontology_term_id', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_type', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length'
    uns: '_scvi', '_training_mode', 'citation', 'dendrogram_cell_type_tissue', 'dendrogram_computational_compartment_assignment', 'de

In [8]:
# Subset to only 10x (exclude Smart-seq2)
adata = adata[adata.obs['assay'] != 'Smart-seq2']

In [9]:
adata

View of AnnData object with n_obs × n_vars = 34092 × 58482
    obs: 'tissue_in_publication', 'assay_ontology_term_id', 'donor_id', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'sex_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'suspension_type', 'cell_type_ontology_term_id', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_type', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length'
    uns: '_scvi', '_training_mode', 'citation', 'dendrogram_cell_type_tissue', 'dendrogram_computational_compartment_assignment', 'de

# Validate obs and uns from adata

In [10]:
merger = AnnDataMerger(
    adata = adata,
    uns_df = uns
)

adata = merger.add_uns_metadata()

adata

Warning: Key title already exists in AnnData UNS. Overwriting with new values.

/home/icb/raphael.kfuri-rubens/miniconda3/envs/hlcav2-env/lib/python3.10/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


AnnData object with n_obs × n_vars = 34092 × 58482
    obs: 'tissue_in_publication', 'assay_ontology_term_id', 'donor_id', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'sex_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'suspension_type', 'cell_type_ontology_term_id', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_type', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length'
    uns: '_scvi', '_training_mode', 'citation', 'dendrogram_cell_type_tissue', 'dendrogram_computational_compartment_assignment', 'dendrogram

In [11]:
# Merge obs and uns
merger = AnnDataMerger(
    adata = adata,
    obs_df = obs
)

cols_to_skip = [
    'organism_ontology_term_id',
    'sex_ontology_term_id',
    'tissue_type',
    'tissue_ontology_term_id',
    'suspension_type',
    'assay_ontology_term_id',
    'is_primary_data',
    'disease_ontology_term_id',
    'self_reported_ethnicity_ontology_term_id'
]

adata_merged = merger.add_obs_metadata(
    adata_col = 'donor_id',
    df_col = 'donor_id',
    skip = cols_to_skip
)

adata_merged.obs

Warning: Column development_stage_ontology_term_id already exists in AnnData OBS. Overwriting with new values.

,tissue_in_publication,assay_ontology_term_id,donor_id,anatomical_information,n_counts_UMIs,n_genes,cell_ontology_class,free_annotation,manually_annotated,compartment,...,cell_number_loaded,sample_collection_year,library_preparation_batch,library_sequencing_run,sequenced_fragment,sequencing_platform,reference_genome,gene_annotation_version,alignment_software,intron_inclusion
AAACCCAAGAACTCCT_TSP14_Lung_Distal_10X_1_1,Lung,EFO:0009922,TSP14,distal,44221.0,6970,type ii pneumocyte,type ii pneumocyte,True,epithelial,...,10000,2020,N/A,N/A,3 prime tag,EFO:0008637,GRCh38,N/A,cell ranger 5.0.1,no
AAACGAAAGGAGCTGT_TSP14_Lung_Distal_10X_1_1,Lung,EFO:0009922,TSP14,distal,28563.0,5638,type ii pneumocyte,type ii pneumocyte,True,epithelial,...,10000,2020,N/A,N/A,3 prime tag,EFO:0008637,GRCh38,N/A,cell ranger 5.0.1,no
AAACGAACACGCGTGT_TSP14_Lung_Distal_10X_1_1,Lung,EFO:0009922,TSP14,distal,4099.0,1929,type ii pneumocyte,type ii pneumocyte,True,epithelial,...,10000,2020,N/A,N/A,3 prime tag,EFO:0008637,GRCh38,N/A,cell ranger 5.0.1,no
AAACGAACATGCCGCA_TSP14_Lung_Distal_10X_1_1,Lung,EFO:0009922,TSP14,distal,24619.0,4021,type ii pneumocyte,type ii pneumocyte,True,epithelial,...,10000,2020,N/A,N/A,3 prime tag,EFO:0008637,GRCh38,N/A,cell ranger 5.0.1,no
AAAGAACGTTAAGGGC_TSP14_Lung_Distal_10X_1_1,Lung,EFO:0009922,TSP14,distal,28129.0,5278,type ii pneumocyte,type ii pneumocyte,True,epithelial,...,10000,2020,N/A,N/A,3 prime tag,EFO:0008637,GRCh38,N/A,cell ranger 5.0.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCAAGCCCG_TSP2_Lung_proxmedialdistal_10X_1_2,Lung,EFO:0009922,TSP2,proxmedialdistal,2892.0,1349,"cd4-positive, alpha-beta t cell","cd4-positive, alpha-beta t cell",True,immune,...,10000,2019,N/A,N/A,3 prime tag,EFO:0008637,GRCh38,N/A,cell ranger 5.0.1,no
TTTGTTGTCGTCAACA_TSP2_Lung_proxmedialdistal_10X_1_2,Lung,EFO:0009922,TSP2,proxmedialdistal,8952.0,3353,basal cell,basal cell,True,epithelial,...,10000,2019,N/A,N/A,3 prime tag,EFO:0008637,GRCh38,N/A,cell ranger 5.0.1,no
TTTGTTGTCTACCACC_TSP2_Lung_proxmedialdistal_10X_1_2,Lung,EFO:0009922,TSP2,proxmedialdistal,3234.0,1566,"cd8-positive, alpha-beta t cell","cd8-positive, alpha-beta t cell",True,immune,...,10000,2019,N/A,N/A,3 prime tag,EFO:0008637,GRCh38,N/A,cell ranger 5.0.1,no
TTTGTTGTCTAGCCAA_TSP2_Lung_proxmedialdistal_10X_1_2,Lung,EFO:0009922,TSP2,proxmedialdistal,2824.0,1380,non-classical monocyte,non-classical monocyte,True,immune,...,10000,2019,N/A,N/A,3 prime tag,EFO:0008637,GRCh38,N/A,cell ranger 5.0.1,no


# Add author cell type markers to UNS

In [12]:
sample_col = [x for x in adata.obs.columns.tolist() if 'cell' in x]
sample_col

['cell_ontology_class',
 'cell_type_ontology_term_id',
 'cell_type',
 'cell_enrichment',
 'cell_viability_percentage',
 'cell_number_loaded']

In [13]:
# No cell type markers available

# Check author cell type annotations and Cell Ontology IDs

In [14]:
adata.obs[AUTHOR_CELL_TYPE] = adata.obs['cell_type']

In [15]:
adata.obs[CELL_TYPE_ONTOLOGY_ID].value_counts()

CL:0000235    12034
CL:0002063     8827
CL:0000646     2055
CL:0000860     1390
CL:0002144     1374
CL:0000158     1023
CL:0000875      999
CL:0002370      729
CL:0000767      677
CL:1000271      567
CL:0000624      524
CL:0000625      523
CL:2000016      480
CL:0002543      478
CL:0000451      278
CL:0002393      252
CL:0000057      244
CL:0002062      204
CL:0000669      195
CL:1000413      161
CL:0000775      138
CL:0001044      132
CL:0000814      126
CL:0000786      125
CL:0000236       82
CL:0001050       80
CL:0002503       74
CL:0000359       71
CL:0002598       67
CL:0000071       47
CL:0002138       36
CL:0000192       25
CL:0017000       19
CL:0000784       17
CL:0000077       16
CL:1000331       13
CL:0000186       10
Name: cell_type_ontology_term_id, dtype: int64

# Check whether ENSEMBL IDs in var

In [16]:
adata.var

,feature_type,highly_variable,means,dispersions,dispersions_norm,mean,std,feature_is_filtered,feature_name,feature_reference,feature_biotype,feature_length
ENSG00000223972,Gene Expression,False,6.398244e-05,0.835044,-0.573947,0.000039,0.005574,False,DDX11L1,NCBITaxon:9606,gene,632
ENSG00000227232,Gene Expression,False,2.274395e-03,2.442280,0.533203,0.001080,0.031731,False,WASH7P,NCBITaxon:9606,gene,1351
ENSG00000278267,Gene Expression,False,6.175251e-05,1.295335,-0.256874,0.000033,0.005634,False,MIR6859-1,NCBITaxon:9606,gene,68
ENSG00000243485,Gene Expression,False,1.372886e-04,2.656352,0.680668,0.000048,0.008041,False,MIR1302-2HG,NCBITaxon:9606,gene,1021
ENSG00000284332,Gene Expression,False,1.000000e-12,NaN,0.000000,0.000000,1.000000,False,MIR1302-2,NCBITaxon:9606,gene,138
...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000198695,Gene Expression,False,9.634841e-01,2.466404,0.154140,0.590065,0.741395,False,MT-ND6,NCBITaxon:9606,gene,525
ENSG00000210194,Gene Expression,False,1.600667e-01,1.603787,-0.044396,0.083929,0.301820,False,MT-TE,NCBITaxon:9606,gene,69
ENSG00000198727,Gene Expression,False,4.367693e+00,4.765751,-0.499747,3.874830,1.104192,False,MT-CYB,NCBITaxon:9606,gene,1141
ENSG00000210195,Gene Expression,False,6.573967e-02,0.624316,-0.719108,0.040580,0.186848,False,MT-TT,NCBITaxon:9606,gene,66


In [17]:
adata.var['ensembl_id'] = adata.var.index
adata.var.index = adata.var['feature_name']
adata.var.index.name = 'index'
adata.var.rename(columns = {'feature_name': 'gene_symbol'}, inplace = True)

adata.var

,feature_type,highly_variable,means,dispersions,dispersions_norm,mean,std,feature_is_filtered,gene_symbol,feature_reference,feature_biotype,feature_length,ensembl_id
index,,,,,,,,,,,,,
DDX11L1,Gene Expression,False,6.398244e-05,0.835044,-0.573947,0.000039,0.005574,False,DDX11L1,NCBITaxon:9606,gene,632,ENSG00000223972
WASH7P,Gene Expression,False,2.274395e-03,2.442280,0.533203,0.001080,0.031731,False,WASH7P,NCBITaxon:9606,gene,1351,ENSG00000227232
MIR6859-1,Gene Expression,False,6.175251e-05,1.295335,-0.256874,0.000033,0.005634,False,MIR6859-1,NCBITaxon:9606,gene,68,ENSG00000278267
MIR1302-2HG,Gene Expression,False,1.372886e-04,2.656352,0.680668,0.000048,0.008041,False,MIR1302-2HG,NCBITaxon:9606,gene,1021,ENSG00000243485
MIR1302-2,Gene Expression,False,1.000000e-12,NaN,0.000000,0.000000,1.000000,False,MIR1302-2,NCBITaxon:9606,gene,138,ENSG00000284332
...,...,...,...,...,...,...,...,...,...,...,...,...,...
MT-ND6,Gene Expression,False,9.634841e-01,2.466404,0.154140,0.590065,0.741395,False,MT-ND6,NCBITaxon:9606,gene,525,ENSG00000198695
MT-TE,Gene Expression,False,1.600667e-01,1.603787,-0.044396,0.083929,0.301820,False,MT-TE,NCBITaxon:9606,gene,69,ENSG00000210194
MT-CYB,Gene Expression,False,4.367693e+00,4.765751,-0.499747,3.874830,1.104192,False,MT-CYB,NCBITaxon:9606,gene,1141,ENSG00000198727


# Check raw data

In [18]:
adata.raw.X.toarray().max()

38165.0

In [19]:
adata.X = adata.raw.X

In [20]:
adata.X.toarray().max()

38165.0

In [21]:
adata.X = adata.X.astype(np.int64)

In [22]:
adata.raw = adata

In [76]:
adata.X.toarray().max()

38165

In [77]:
adata.raw.X.toarray().max()

38165

## Validation result

### UNS Validation
- OK: Tier 1 UNS Google Sheet
- OK: Tier 1 UNS AnnData Object

### OBS Validation
- OK: Tier 1 OBS Google Sheet
- OK: Tier 1 OBS AnnData Object

# Data Submission Status

- CHECK: Raw data in X and in raw
- CHECK: Tier 1 Metadata in OBS
- CHECK: Cell Ontology IDs in OBS
- CHECK: Author cell type in OBS
- MISSING: Marker genes in UNS
- CHECK: ENSEMBL IDs and gene symbols in var

Marker genes not submitted. Not retrievable from publication upon first search. Check again.

In [78]:
adata.obs['cell_viability_percentage'] = pd.to_numeric(adata.obs['cell_viability_percentage'], errors='coerce')
adata.obs['cell_number_loaded'] = pd.to_numeric(adata.obs['cell_number_loaded'], errors='coerce')
adata.obs['sample_collection_year'] = pd.to_numeric(adata.obs['sample_collection_year'], errors='coerce')

In [79]:
adata.write_h5ad(join(OUTPUT_PATH_PREREVISION, f"{DATASET_ID}.h5ad"), compression='gzip')
adata.write_h5ad(join(OUTPUT_PATH_POSTREVISION, f"{DATASET_ID}.h5ad"), compression='gzip')